In [28]:
!pip install pytorch-crf
!pip install unidecode

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install --upgrade torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [31]:
import numpy as np
import pandas as pd
import os
import string
from unidecode import unidecode
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import AdamW, BertModel, AutoConfig, AutoTokenizer
from torch.optim import Adam
import torch.nn as nn
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
import time;
import datetime
from sklearn.model_selection import train_test_split
from torchcrf import CRF

In [32]:
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [33]:
# nlp_train=pd.read_csv("/kaggle/input/nlp-project-train/train.csv", index_col=[0])
# nlp_test=pd.read_csv("/kaggle/input/nlp-project-train/test.csv",index_col=[0],encoding="windows-1252") 

nlp_train=pd.read_csv("train.csv", index_col=[0])
nlp_test=pd.read_csv("test.csv",index_col=[0],encoding="windows-1252") 

In [34]:
def convert_to_ascii(sentence):
    text = unidecode(sentence)
    return text

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        text = text.replace('  ', ' ')
    return text.strip()

def split_sentences(sentences, max_length=100):
    punctuations = {'.', ',', ';', ':'}
    results = []

    for sentence in sentences:
        while len(sentence) > max_length:
            # find last punctuation before max_length
            split_pos = -1
            for p in punctuations:
                pos = sentence.rfind(p, 0, max_length + 1)
                if pos > split_pos:
                    split_pos = pos
            
            # If no punctuation found, split at the last space before max_length
            if split_pos == -1:
                split_pos = sentence.rfind(' ', 0, max_length + 1)
            
            # If no space found, just split at max_length
            if split_pos == -1:
                split_pos = max_length
            
            # Append the split segment to results
            results.append(sentence[:split_pos + 1].strip())
            # Move the rest of the sentence forward
            sentence = sentence[split_pos + 1:].strip()
        
        # Append the remainder of the sentence if it's not empty
        if sentence:
            results.append(sentence)
    
    return np.array(results)


In [35]:
# Raw sentences
raw_sentences = nlp_train["Sentence"].values
# Split sentences
sentences = split_sentences(raw_sentences, max_length=200)
# Remove punctuations
sentences = [remove_punctuations(s).lower() for s in sentences]
# Apply convert to ascii to y_train
asci_sentences = [convert_to_ascii(s) for s in sentences]

In [36]:
def encode_word(sentence):
    for char in sentence:
        # print(char)
        if char in "ıöüğşç":
            # replace char with 2
            sentence = sentence.replace(char, '2')
        else:
            sentence = sentence.replace(char, '1')
    return sentence

In [37]:
sentences_diacritics = sentences.copy()
for sentence_index in range(len(sentences_diacritics)):
    sentence = sentences_diacritics[sentence_index]
    new_sentence = encode_word(sentence)
    sentences_diacritics[sentence_index] = new_sentence

In [38]:
sentences_diacritics[0]

'121211111111111122111111112112211111211111211121111111121111111211212111111111111111112111112111'

In [39]:
def align_tokens_with_diacritics(texts, diacritics, tokenizer, label_dict):
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_offsets_mapping=True, return_tensors="pt")
    input_ids = tokenized_texts['input_ids']
    attention_masks = tokenized_texts['attention_mask']
    token_type_ids = tokenized_texts['token_type_ids']
    offset_mappings = tokenized_texts['offset_mapping']
    
    # Initialize the labels for each token to a default numeric value, e.g., label_dict['O'] for 'Other'
    labels = [[label_dict['O']] * len(input_id) for input_id in input_ids]

    for i, offsets in enumerate(offset_mappings):
        for token_index, (start, end) in enumerate(offsets):
            if start == end:  # Special tokens, skip them
                continue
            # Extract the original character sequence corresponding to the current token
            char_sequence = texts[i][start:end]
            # Assuming diacritics are stored in a similar structure as texts
            corresponding_diacritic = diacritics[i][start:end]
            # Assign the diacritic label to the first character's diacritic or any suitable logic
            # Use the dictionary to convert string labels to integers
            labels[i][token_index] = label_dict[corresponding_diacritic[0]] if corresponding_diacritic else label_dict['O']

    return input_ids, attention_masks, token_type_ids, labels


In [40]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
label_dict = {'O': 0, '1': 1, '2': 2}
input_ids, attention_masks, token_type_ids, labels = align_tokens_with_diacritics(
    asci_sentences, sentences_diacritics, tokenizer, label_dict
)

In [41]:
# Convert lists of labels into a tensor; assuming each label is already converted to a numeric form
label_tensors = torch.tensor(labels, dtype=torch.long)

# Create a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, token_type_ids, label_tensors)

# You can now use this dataset with a DataLoader to create batches
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [42]:
from transformers import BertForTokenClassification, AdamW

# Assuming you have a specific number of diacritic labels
num_labels = len(set([label for sublist in labels for label in sublist]))  # Adjust this as necessary

model = BertForTokenClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=False,
)

# Move model to GPU if available
model.to(device)

# Setting up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/mustafa/miniconda3/envs/nlp_project/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [43]:
from tqdm import tqdm

epochs = 1

# model.train()
# for epoch in range(epochs):
#     total_loss = 0
#     for step, batch in enumerate(tqdm(dataloader)):
#         batch = tuple(t.to(device) for t in batch)
#         b_input_ids, b_input_mask, b_token_types, b_labels = batch
#         model.zero_grad()
        
#         outputs = model(b_input_ids, token_type_ids=b_token_types, attention_mask=b_input_mask, labels=b_labels)
#         loss = outputs[0]
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch}, Loss: {total_loss / len(dataloader)}")

In [44]:
model = torch.load('full_model.pth', map_location=torch.device('cpu'))
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [47]:
# Input text
input_text = "sinif havuz ve acik deniz calismalariyla tum dunyada"

# Tokenize the input
test_inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
test_inputs = {k: v.to(device) for k, v in test_inputs.items()}

# Predict
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Get tokens for reference
tokens = tokenizer.convert_ids_to_tokens(test_inputs['input_ids'][0])

# Define diacritics mapping
diacritics_mapping = {
    'c': {2: 'ç'},
    'g': {2: 'ğ'},
    'i': {2: 'ı'},
    'o': {2: 'ö'},
    's': {2: 'ş'},
    'u': {2: 'ü'}
}

# Function to apply diacritics based on predictions
def apply_diacritics(tokens, predictions, mapping):
    diacritized_text = []
    skip_tokens = ['[CLS]', '[SEP]', '[PAD]']  # Tokens to skip when applying diacritics

    # Accumulator for reassembling subword tokens
    current_word = ""
    for token, pred in zip(tokens, predictions[0]):
        if token in skip_tokens:
            continue

        # Check if the token is a continuation of the previous one
        if token.startswith("##"):
            current_word += token[2:]  # Append without "##" and without adding space
        else:
            # If there's accumulated word data from previous tokens, append it first
            if current_word:
                diacritized_text.append(current_word)
                current_word = ""
            # Start a new word accumulation
            current_word = token

        # Apply diacritics to the current part of the word if applicable
        if current_word in mapping and pred.item() in mapping[current_word]:
            # Replace the whole word with its diacritized version
            current_word = mapping[current_word][pred.item()]

    # Append the last accumulated word if any
    if current_word:
        diacritized_text.append(current_word)

    return " ".join(diacritized_text)

# Apply diacritics
diacritized_output = apply_diacritics(tokens, predictions, diacritics_mapping)
print(predictions)
print("Original:", input_text)
print("Diacritized:", diacritized_output)

tensor([[0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0]])
Original: sınif havuz ve acik deniz calismalariyla tum dunyada
Diacritized: sınif havuz ve acik deniz calismalariyla tum dunyada
